In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from sklearn import metrics
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from datetime import datetime
import cv2
from collections import OrderedDict

from tqdm import tqdm
from joblib import Parallel

In [13]:
import pickle

In [3]:
PATH = Path("/data2/yinterian/multi-task-romain")

In [4]:
data = pd.read_csv(PATH/"validation_jan_2020/data_validation_5min_gap.csv")

In [5]:
data.head()

,subject_id,key,gender,age,sapsii,sofa,care_unit,amine,sedation,ventilation,...,time_and_date,hr,spo2,abp_sys,abp_dias,abp_mean,observation_mean_MAP,prediction_mean_MAP,observation_mean_HR,prediction_mean_HR
0,3,3_1,0,82,122,16,5,1,1,1,...,2017-11-16 09:10:00,93,100,146,60,83,75.766667,78.6,88.866667,91.0
1,3,3_1,0,82,122,16,5,1,1,1,...,2017-11-16 09:11:00,89,100,139,56,77,75.766667,78.6,88.866667,91.0
2,3,3_1,0,82,122,16,5,1,1,1,...,2017-11-16 09:12:00,89,100,133,54,73,75.766667,78.6,88.866667,91.0
3,3,3_1,0,82,122,16,5,1,1,1,...,2017-11-16 09:13:00,88,100,141,57,78,75.766667,78.6,88.866667,91.0
4,3,3_1,0,82,122,16,5,1,1,1,...,2017-11-16 09:14:00,90,100,135,55,74,75.766667,78.6,88.866667,91.0


In [6]:
period = data[data["key"] == "3_1"]

In [8]:
def process_periods(df, index):
    time_cols = ["hr", "spo2", "abp_sys", "abp_dias", "abp_mean"]
    static_cols = ["subject_id", "key", "gender", "age", "sapsii", "sofa", "care_unit", "amine", "sedation",\
                   "ventilation"]
    y_cols = ["prediction_mean_HR", "prediction_mean_MAP"]
    static_df = df.loc[:, static_cols + y_cols].iloc[0]
    time_df = df.loc[:, time_cols]
    time_series = [time_df.iloc[i,].values for i in range(time_df.shape[0])]
    static_df["series"] = time_series
    return static_df.copy().to_frame(index).T

In [9]:
def process_whole_list(obs):
    list_df = []
    obs_periods = np.unique(obs.key.values)
    for i in tqdm(range(len(obs_periods))):
        key = obs_periods[i]
        period = obs[obs["key"] == key]
        line = process_periods(period, i)
        list_df.append(line)
    return list_df

In [10]:
def create_dataset(filename1, filename2):
    data = pd.read_csv(PATH/filename1)
    print(filename1, data.shape)
    data = data[data["window"] == "obs"]
    print(data.shape)
    list_obs = process_whole_list(data)
    data_df = pd.concat(list_obs)
    print(filename2)
    with open(PATH/filename2, 'wb') as f:
        pickle.dump(data_df, f)

In [14]:
data = pd.read_csv(PATH/"validation_jan_2020/data_validation_5min_gap.csv")

In [15]:
create_dataset("validation_jan_2020/data_validation_5min_gap.csv", "data_validation_5min.pickle")

  1%|          | 10/1597 [00:00<00:16, 97.44it/s]

validation_jan_2020/data_validation_5min_gap.csv (63880, 22)
(47910, 22)


100%|██████████| 1597/1597 [00:16<00:00, 96.36it/s]


data_validation_5min.pickle


In [16]:
create_dataset("validation_jan_2020/data_validation_10min_gap.csv", "data_validation_10min.pickle")

  1%|          | 11/1314 [00:00<00:12, 102.93it/s]

validation_jan_2020/data_validation_10min_gap.csv (59130, 22)
(39420, 22)


100%|██████████| 1314/1314 [00:13<00:00, 100.57it/s]


data_validation_10min.pickle


In [17]:
create_dataset("validation_jan_2020/data_validation_15min_gap.csv", "data_validation_15min.pickle")

  1%|          | 11/1097 [00:00<00:10, 105.52it/s]

validation_jan_2020/data_validation_15min_gap.csv (54850, 22)
(32910, 22)


100%|██████████| 1097/1097 [00:10<00:00, 103.40it/s]


data_validation_15min.pickle
